In [3]:
# Creating a SparkSession

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("Repartitioning_Joins_PySpark").getOrCreate()

In [5]:
# Emp Data & Schema

emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","Female","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

In [6]:
emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

In [7]:
dept_data = [
    ["101", "Sales", "NYC", "US", "1000000"],
    ["102", "Marketing", "LA", "US", "900000"],
    ["103", "Finance", "London", "UK", "1200000"],
    ["104", "Engineering", "Beijing", "China", "1500000"],
    ["105", "Human Resources", "Tokyo", "Japan", "800000"],
    ["106", "Research and Development", "Perth", "Australia", "1100000"],
    ["107", "Customer Service", "Sydney", "Australia", "950000"]
]

dept_schema = "department_id string, department_name string, city string, country string, budget string"

In [8]:
# Create emp DataFrame

emp = spark.createDataFrame(data=emp_data, schema=emp_schema)
dept = spark.createDataFrame(data=dept_data, schema=dept_schema)

In [9]:
emp.show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

In [10]:
dept.show()

+-------------+--------------------+-------+---------+-------+
|department_id|     department_name|   city|  country| budget|
+-------------+--------------------+-------+---------+-------+
|          101|               Sales|    NYC|       US|1000000|
|          102|           Marketing|     LA|       US| 900000|
|          103|             Finance| London|       UK|1200000|
|          104|         Engineering|Beijing|    China|1500000|
|          105|     Human Resources|  Tokyo|    Japan| 800000|
|          106|Research and Deve...|  Perth|Australia|1100000|
|          107|    Customer Service| Sydney|Australia| 950000|
+-------------+--------------------+-------+---------+-------+



In [11]:
emp.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)



In [13]:
dept.printSchema()

root
 |-- department_id: string (nullable = true)
 |-- department_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- budget: string (nullable = true)



In [14]:
# Get number of partitions for emp
emp.rdd.getNumPartitions()

8

In [15]:
dept.rdd.getNumPartitions()

8

In [16]:
# Repartition of data using repartition & coalesce
emp_partitioned = emp.repartition(4)
emp_partitioned.rdd.getNumPartitions()

4

In [17]:
# Find the partition info for partitions and repartition
from pyspark.sql.functions import spark_partition_id

emp_1 = emp.repartition(4, "department_id").withColumn("partition_num", spark_partition_id())
emp_1.show()

+-----------+-------------+-------------+---+------+------+----------+-------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|partition_num|
+-----------+-------------+-------------+---+------+------+----------+-------------+
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|            0|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|            0|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|            0|
|        014|          107|    Emily Lee| 26|Female| 46000|2019-01-01|            0|
|        016|          107|  Kelly Zhang| 30|Female| 49000|2018-04-01|            0|
|        020|          102|    Grace Kim| 32|Female| 53000|2018-11-01|            0|
|        012|          105|   Susan Chen| 31|Female| 54000|2017-02-15|            1|
|        017|          105|  George Wang| 34|  Male| 57000|2016-03-15|            1|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-0

## Joins

In [20]:
# INNER JOIN
# select e.emp_name, d.depart_name, d.department_id, e.salary from emp e inner join dept d on emp.department_id = dept.department_id

df_joined = emp.join(dept, how="inner", on=emp.department_id==dept.department_id)
df_joined.select(emp.name, dept.department_id, dept.department_name, emp.salary).show()

+-------------+-------------+--------------------+------+
|         name|department_id|     department_name|salary|
+-------------+-------------+--------------------+------+
|     John Doe|          101|               Sales| 50000|
|   Jane Smith|          101|               Sales| 45000|
|James Johnson|          101|               Sales| 70000|
|    Bob Brown|          102|           Marketing| 55000|
|    Alice Lee|          102|           Marketing| 48000|
|     Kate Kim|          102|           Marketing| 51000|
|    Grace Kim|          102|           Marketing| 53000|
|    Jack Chan|          103|             Finance| 60000|
|    Jill Wong|          103|             Finance| 52000|
|      Tom Tan|          103|             Finance| 58000|
|  Steven Chen|          103|             Finance| 62000|
|     Lisa Lee|          104|         Engineering| 47000|
|   David Park|          104|         Engineering| 65000|
|    Nancy Liu|          104|         Engineering| 50000|
|   Susan Chen